# Entitlement API (Sample)


**This is a sample code to achieve all entitlement API functionalities. Use it as a reference to implement your API."

Define a class "Eapi" to achieve all API requests:

* Eapi
  * .client_id: store client id
  * .client_secret: store client secret
  * .run_host: host to call
  * .jwt: jwt to create
  * .ctrb: path url to create
  * .gen_jwt(): method to generate jwt
  * .get_pkg_path(): method to create path url for pkg request
  * .pkg(): call package type request
  * .get_tkt_path(): method to create path url for tkt request
  * .tkt(): call ticket type request
  
Click [here](#Eapi-class) to see the detail of the code at the very end.

In [ ]:
# Import libraries, jwt is not a default library for python, you may install it via pip.
import jwt

import base64
import binascii
import datetime
import requests
import json
import pandas as pd

In [ ]:
# input credentials and connection mode

client_id = ""
client_secret = ""

host_option = {"BETA": "https://beta.api.bloomberg.com",
              "PROD": "https://api.bloomberg.com"}

host_select = "PROD"

In [ ]:
# Initialize/refresh the Eapi instance

prod_test = Eapi(client_id, client_secret, host_option[host_select])

In [ ]:
# Claim data request, example below

pkg_one_user1 = {
            "email": "dliu279@gmail.com",
            "packageIds": [
                508421, 508420
            ],
            "status": "entitled",
            "uuid": 21278362
        }

pkg_one_user2 = {
            "email": "cyu3@gmail.com",
            "packageIds": [
                508421
            ],
            "status": "entitled",
            "uuid": 515555
        }

pkg_req = {}
pkg_req["updates"] = [pkg_one_user1, pkg_one_user2]
# pkg_req_data = json.dumps(pkg_req)  #set bulk update list
pkg_req_data = ""  #set as empty if not PATCH request

In [ ]:
# Ticket type request

ticket_id = ""
status = ""
start_date = ""
end_date = ""
tkt_act = "GET" #GET, PATCH
data = ""

res, result = prod_test.tkt(tkt_act, ticket_id, status, start_date, end_date)

print(res)    #check response status

In [ ]:
result['tickets'][0]   #check ticket details

In [ ]:
# Package type request

pkg_id = ""
user_id = ""
email = "nwang166@bloomberg.net"
pkg_act = "GET" #GET, PATCH
pkg_data = pkg_req_data

res, result = prod_test.pkg(pkg_act, pkg_id, user_id, email, pkg_data)

print(res)    #check response status

In [ ]:
# Check the user information
result['users'][0]['userInfo']

In [ ]:
# Check entitlement for all packages
pd.DataFrame.from_dict(result['users'][0]['packageInfo'])

## Eapi class

**Important tips:**
* Eapi.broker_id is manually set, please check with your Bloomberg rep to get this;
* "iat", "nbf", and "exp" are defined in a way as an example here, you may change it for your need but still must follow rules in entitlement documentation.

In [ ]:
class Eapi:
    
    def __init__(self, client_id, client_secret, run_host="https://beta.api.bloomberg.com"):
        
        self.client_id = client_id
        self.client_secret = client_secret
        self.run_host = run_host
        self.jwt = ""
        self.ctrb = ""
        
        self.broker_id = 894 #pre setup
        
    
    def gen_jwt(self, jwt_action, jwt_path):
        
        created_date = datetime.datetime.now()
        mid_date = created_date
        exp_date = created_date + datetime.timedelta(seconds=15)

        iat = int(round(created_date.timestamp()))
        nbf = int(round(mid_date.timestamp()))
        exp = int(round(exp_date.timestamp()))
        
        self.credentials = {
                            "client_id": self.client_id,
                            "client_secret": self.client_secret,
                            "expiration_date":created_date
        }
        
        self.jwt_header = {
                            "alg": "HS256",
                            "typ": "JWT"
        }
        
        self.jwt_payload = {
            "iss": self.client_id,
            "exp": exp,
            "nbf": nbf,
            "iat": iat,
            "region": "ny",
            "method": jwt_action,
            "path": jwt_path, #jwt_path
            "host": self.run_host,
            "client_id": self.client_id
        #     "request_id": "9c5b34ce-4196-447c-9008-536150575978"
        }
        
    
        binary_secret = binascii.unhexlify(self.client_secret)
        return jwt.encode(self.jwt_payload, binary_secret, algorithm="HS256")
    
    
    def get_pkg_path(self, pkg_id, user_id, email):
        
        if len(pkg_id)>0:
            p = "/{}".format(pkg_id)
        else:
            p = ""
            
        if len(user_id)>0:
            u = "/{}".format(user_id)
        else:
            u = ""
            
        jwt_path = "/ctrb/v1/packages" + p + "/users"
        
        
        temp_path = ""
        
        if len(user_id)>0:
            temp_path += ("&uuids=" + str(user_id))
        
        if len(email)>0:
            temp_path += ("&emails=" + email)
        
        ctrb_path = jwt_path + "?brokerId=" + str(self.broker_id) + temp_path
        
        
        return ctrb_path, jwt_path
    
    
    def pkg(self, pkg_act="GET", pkg_id="", user_id="", email="", pkg_data=""):
        
        ctrb_path, jwt_path = self.get_pkg_path(pkg_id, user_id, email)
        
        self.ctrb = ctrb_path
        self.jwt = self.gen_jwt(pkg_act, jwt_path).decode()
        
        url = self.run_host + self.ctrb + "&jwt=" + self.jwt
        
        self.url = url
        
        if pkg_act=="GET":
            response = requests.get(url)
        else:
            response = requests.patch(url, data=pkg_data)
            pass
        
        return response, response.json()
    
    
    def get_tkt_path(self, ticket_id, status, start_date, end_date):
        
        if len(ticket_id)>0:
            tk = "/{}".format(ticket_id)
        else:
            tk = ""
            
            
        jwt_path = "/ctrb/v1/tickets" + tk
        
        
        temp_path = ""
        
        if len(status)>0:
            temp_path += ("&status=" + status)
        
        if len(start_date)>0:
            temp_path += ("&startDate=" + start_date)
            
        if len(end_date)>0:
            temp_path += ("&endDate=" + end_date)
        
        ctrb_path = jwt_path + "?brokerId=" + str(self.broker_id) + temp_path
        
        
        return ctrb_path, jwt_path
    
    
    def tkt(self, tkt_act="GET", ticket_id="", status="pending", start_date="", end_date=""):
        
        ctrb_path, jwt_path = self.get_tkt_path(ticket_id, status, start_date, end_date)
        
        self.ctrb = ctrb_path
        self.jwt = self.gen_jwt(tkt_act, jwt_path).decode()
        
        url = self.run_host + self.ctrb + "&jwt=" + self.jwt
        
        self.url = url
        
        if tkt_act=="GET":
            response = requests.get(url)
        else:
            pass
        
        return response, response.json()